In [34]:
# Config
import os

# Basic
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Monitoring
from tqdm.notebook import tqdm

# IO
from os.path import join, exists, basename, dirname, splitext, expanduser
from glob import glob

# Parallel processing
from joblib import Parallel, delayed

import re
from PIL import Image
import supervision as sv
from supervision.metrics import MeanAveragePrecision
from supervision.metrics import MetricTarget
from tempfile import mkdtemp

from ultralytics import YOLO


In [35]:
home_dir="/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/"
base_state = "uttar_pradesh" ## state for which model is trained
target_state = "bihar" ## state for which model is to be validated
test_state = "bihar" ## state for which model is to be validatd during training
model_name = "yolo11m-obb.pt" ## trained model name
task="obb" ## task for which model is trained
conf=0.001 ## confidence threshold
iou=0.33 ## iou threshold
ratio=0.25

# base_dir="/home/patel_zeel/kiln_compass_24/data/"

gt_image_dir= f"../data/region_performace/{target_state}_{ratio}/images"
gt_label_dir=f"../data/region_performace/{target_state}_{ratio}/labels"
print(gt_image_dir)
print(gt_label_dir)

## class information in yml file 
data_yml_save_path=mkdtemp()
data_yml = f"""train: dummy
val: dummy
nc: 3
names: ["CFCBK", "FCBK", "Zigzag"]
"""
data_yml_path = f"{data_yml_save_path}/data.yml"
with open(data_yml_path, "w") as f:
    f.write(data_yml)

print(data_yml_path)  


# "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/train_bihar_test_haryana_0.25_obb_v1_yolo11m-obb.pt_640_128_300
# model_path="/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/train_haryana_test_haryana_0.25_obb_v1_yolo11m-obb.pt_640_128_300/weights/best.pt"
## model path
yolo_runs_dir = "/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/"
model_dir= f"{yolo_runs_dir}train_{base_state}_test_{test_state}_{ratio}_{task}_v1_{model_name}_640_128_300/weights/"
model_path = f"{model_dir}best.pt"
print(model_path)

## output path
result_dir=f"{yolo_runs_dir}train_{base_state}_test_{target_state}_{ratio}_{task}_v1_{model_name}_640_128_300_{conf}_{iou}/"
# prediction_dir = join(result_dir,"labels")
# # print(result_dir)
# print(prediction_dir)


../data/region_performace/bihar_0.25/images
../data/region_performace/bihar_0.25/labels
/tmp/tmpcddqa6ev/data.yml
/home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/train_uttar_pradesh_test_bihar_0.25_obb_v1_yolo11m-obb.pt_640_128_300/weights/best.pt


In [36]:
## Supervision dataset
sv_dataset=sv.DetectionDataset.from_yolo(gt_image_dir,gt_label_dir,data_yml_path)
print(sv_dataset)
print(len(sv_dataset))
sv_dataset.classes

## Yolo model
model = YOLO(model_path)

731


In [37]:
targets=[]
predictions=[]

for name,_,gt_detection in tqdm(sv_dataset):
    file_name=splitext(basename(name))[0]
    # prediction_path=join(prediction_dir,f"{file_name}.txt")
    ultralytics_result=model(name,imgsz=640,iou=0.33,conf=0.001,exist_ok=True,save_txt=True,max_det=300,name=result_dir, verbose=False)[0]
    sv_detection=sv.Detections.from_ultralytics(ultralytics_result)
    targets.append(gt_detection)
    predictions.append(sv_detection)









  0%|          | 0/731 [00:00<?, ?it/s]

Results saved to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/train_uttar_pradesh_test_bihar_0.25_obb_v1_yolo11m-obb.pt_640_128_300_0.001_0.33
1 label saved to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/train_uttar_pradesh_test_bihar_0.25_obb_v1_yolo11m-obb.pt_640_128_300_0.001_0.33/labels
Results saved to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/train_uttar_pradesh_test_bihar_0.25_obb_v1_yolo11m-obb.pt_640_128_300_0.001_0.33
2 labels saved to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/train_uttar_pradesh_test_bihar_0.25_obb_v1_yolo11m-obb.pt_640_128_300_0.001_0.33/labels
Results saved to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/train_uttar_pradesh_test_bihar_0.25_obb_v1_yolo11m-obb.pt_640_128_300_0.001_0.33
3 labels saved to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/runs/obb/train_uttar_pradesh_test_bihar_0.25_obb_v1_yolo11m-obb.pt_640_128_300_0.001

In [38]:
## mAP calculation 
mAP_metric = MeanAveragePrecision(class_agnostic=False)

mAP_result=mAP_metric.update(predictions,targets).compute()
class_wise_mAP=mAP_result.ap_per_class[:,0].tolist()
# print(f"mAP_result: {mAP_result}")
matched_classes=mAP_result.matched_classes.tolist()
print(f"Matched classes: {matched_classes}")
num_classes=3
final_class_wise_mAP = [0]*num_classes

for cls, mAP in zip(matched_classes, class_wise_mAP):
    print(f"cls: {cls}, mAP: {mAP}")
    final_class_wise_mAP[cls] = mAP


print(f"class_wise_mAP: {final_class_wise_mAP}")


mAP_metric = MeanAveragePrecision(class_agnostic=True)
mAP_result = mAP_metric.update(predictions, targets).compute()
class_agnostic_result = mAP_result.ap_per_class[:, 0].tolist()
# print(f"mAP_result: {mAP_result}")
print(f"class_wise_mAP: {class_wise_mAP}")
print(f"class_agnostic_result: {class_agnostic_result}")

Matched classes: [1, 2]
cls: 1, mAP: 0.04047346966129055
cls: 2, mAP: 0.3540466832839704
class_wise_mAP: [0, 0.04047346966129055, 0.3540466832839704]
class_wise_mAP: [0.04047346966129055, 0.3540466832839704]
class_agnostic_result: [0.4729578500570446]


In [39]:
index = pd.MultiIndex.from_tuples([], names=["Base State", "Target State"])
result_df = pd.DataFrame(columns=["CFCBK", "FCBK", "Zigzag", "mAP", "Class-agnostic AP"], index=index)
result_df.loc[(base_state,target_state),:] = final_class_wise_mAP + [mAP_result.mAP_scores[0]] + class_agnostic_result
display(result_df)

,,CFCBK,FCBK,Zigzag,mAP,Class-agnostic AP
Base State,Target State,,,,,
uttar_pradesh,bihar,0,0.040473,0.354047,0.472958,0.472958


In [40]:
output_csv_path = f"{home_dir}result_csv/different_state_to_{target_state}_mAP_results.csv"


In [41]:
output_csv_path = f"{home_dir}result_csv/different_state_to_{target_state}_mAP_results.csv"
os.makedirs(dirname(output_csv_path), exist_ok=True)


if not os.path.exists(output_csv_path):
    result_df.to_csv(output_csv_path, header=True)
    print(f"Results saved to {output_csv_path}")
else:
    result_df.to_csv(output_csv_path, mode='a', header=False)
    print(f"Results appended to {output_csv_path}")

Results appended to /home/rishabh.mondal/Brick-Kilns-project/ijcai_2025_kilns/result_csv/different_state_to_bihar_mAP_results.csv


In [9]:
base_state_2 =  ## state for which model is trained
output_csv_path_2 = f"{home_dir}result_csv/{base_state_2}_to_different_state_mAP_results.csv"

result_west_bengal=pd.read_csv(output_csv_path)
result_punjab=pd.read_csv(output_csv_path_2)
result=pd.concat([result_west_bengal,result_punjab],ignore_index=True)
display(result)

SyntaxError: invalid syntax (1003467514.py, line 1)

In [45]:
csv_path=f"../result_csv/different_state_to_bihar_mAP_results.csv"
result=pd.read_csv(csv_path)
display(result.style.background_gradient(cmap='viridis',axis=0))

,Base State,Target State,CFCBK,FCBK,Zigzag,mAP,Class-agnostic AP
0,bihar,bihar,0,0.790966,0.820665,0.811288,0.811288
1,punjab,bihar,0,0.000323,0.383721,0.373844,0.373844
2,west_bengal,bihar,0,0.018392,0.415290,0.434204,0.434204
3,haryana,bihar,0,0.013614,0.436447,0.410338,0.410338
4,uttar_pradesh,bihar,0,0.040473,0.354047,0.472958,0.472958


In [29]:
def apply_state_color(val):
    if val == 'West Bengal':
        return 'background-color: yellow'  # color for West Bengal (Yellow)
    elif val == 'Punjab':
        return 'background-color: green'  # color for Punjab (Green)
    else:
        return ''  # no color if it's other

# Apply the color styling for 'Base State' and 'Target State' columns
styled_result = result.style\
    .applymap(apply_state_color, subset=["Base State", "Target State"])\
    .background_gradient(subset=["CFCBK", "FCBK", "Zigzag", "mAP", "Class-agnostic AP"], cmap="coolwarm")\
    .set_properties(**{"text-align": "center"})\
    .set_table_styles([
        {"selector": "th", "props": [("font-weight", "bold"), ("text-align", "center")]},
        {"selector": "tr:nth-child(even)", "props": [("background-color", "#f9f9f9")]}
    ])

# Displaying the styled DataFrame
display(styled_result)

,Base State,Target State,CFCBK,FCBK,Zigzag,mAP,Class-agnostic AP
0,west_bengal,punjab,0.000000,0.019683,0.425855,0.492828,0.492828
1,west_bengal,uttar_pradesh,0.000000,0.129238,0.197902,0.417703,0.417703
2,west_bengal,haryana,0.000000,0.004189,0.236194,0.270739,0.270739
3,west_bengal,bihar,0.000000,0.033873,0.412915,0.451084,0.451084
4,punjab,haryana,0.000000,0.008151,0.643177,0.659907,0.659907
5,punjab,bihar,0.000000,0.012281,0.338602,0.322256,0.322256
6,punjab,uttar_pradesh,0.000000,0.035343,0.137244,0.215012,0.215012
7,punjab,west_bengal,0.000000,0.016273,0.133746,0.199013,0.199013
